In [1]:
from PyPDF2 import PdfReader
import re
import pandas as pd

In [4]:
#This coding is suitable for TSMC

def clean_str(s):
    s=re.sub(r'\n| ','',s)

    if "：" or r'\u3000' in s:
        s=re.sub(r"：|\u3000",'',s)

    return s

def rev_visitor_body(text, cm, tm, fontDict, fontSize):

    global rev_found,cur,subtitle,category,t,p,ti,temp_p,time,temp_c,finish,mn,money
    print(text)
    if not '收入' in text and not rev_found: return None
    s=clean_str(text)

    print(s,rev_found)

    if rev_found:pass
    elif not '收入之細分' in s : return None# or not '收入' in s  : return None
    else:
        rev_found=True
        print('Found')
        return ''

    title_pattern=r'[\(一二三四五六七八九十\)]{3,}、{0,1}|[0-9]{1,}\.[\u4e00-\u9fff]+'
    if re.search(title_pattern,text) and rev_found:
        print('Got off')
        category.append(temp_c)
        mn.append(money)
        rev_found=False 
        finish=True
        return ''

    
    #if not (tm[4],tm[5],text) == (0.0, 0.0, ''):
    #if not int(tm[4])==0:parts.append((tm[4],tm[5],text))

    print('Text--------------\n',tm[4],tm[5],text)
    if not cur:
        # Locate the revenue part
        print(money,text)
        if not '$' in text:
            time.append(text)    
            
        else:
            print('Here----------------',''.join(time))
            print('right now text',text)
            cur = True
            time_pattern=r'[0-9]{3}年[0-9]{1,2}月([0-9]{1,2}日){0,1}(至[0-9]{1,2}月){0,}([0-9]{1,2}日){0,}|[0-9]{3}\.[0-9]{1,2}\.[0-9]{1,2}'            
            time_str=''.join(time)
            time_str=clean_str(time_str)

            ms=list(re.finditer(time_pattern,time_str))
            print('Check my subtitle',ms)
            subtitle.append(time_str[:ms[0].start()])
            if temp_c!=[]:
                category.append(temp_c)
                mn.append(money)
                temp_c=[]
                money=[]              

            time_window.append([m.group(0) for m in ms])
            #category.append(time_str[ms[-1].end():])

            t=[(1,2,time_str[ms[-1].end():]),(tm[4],tm[5],text)]
            print('Cur just found-----',t)
    
    if t!=[]:
        print('compare:',(tm[5],t[-1][1]),(tm[4],t[-1][0]))
        print('show current t',t)

        if int(tm[4])==0:pass
        elif abs(tm[5]-t[-1][1])<1.5 and tm[4]>=t[-1][0]:
            t.append((tm[4],tm[5],text))

        else:

            #Deal with detailed data, such as revenue subject with fininancial numbers
            temp_p.append(''.join([txt[2] for txt in t]))

            if tm[4]<t[-1][0]:
                print('temp-p',temp_p)
                info=clean_str(''.join(temp_p))
                print('info--',info)
                money_pattern=r'([0-9]{1,3},){0,}[0-9]{1,3}|-'
                #sumup_pattern=r'\$([0-9]{1,3},){0,}[0-9]{1,3}[\u4e00-\u9fff]{0,}|-[\u4e00-\u9fff]{0,}'
                ms=list(re.finditer(money_pattern,info))
                print('re finding,',ms)
                
                if ms==[]:
                    temp_p=[]
                    return ''

                if info[:ms[0].start()]=='$' and not re.search(r'[\u4e00-\u9fff]+',info): 
                    cur = False
                    time=[]
                    temp_p=[]
                    t=[]
                    #category.append()

                    print('End of a section...')
                    print(time,t)

                    return ''
                else:

                    # In case we fetch the sumup, then we can only tell by chinese character
                    # Detect the chinese character in the middle of the string
                    cc=list(re.finditer(r'[\u4e00-\u9fff]+',info))
                    if not len(cc)==0:
                        temp_c.append(info[:cc[0].end()])
                        info=info[cc[0].end():]
                    ms=list(re.finditer(money_pattern,info))
                    temp_m=[]
                    for m in ms:
                        temp_m.append(m.group(0))
                    money.append(temp_m)

                temp_p=[]

            t=[(tm[4],tm[5],text)]


    '''
    if parts[i][1]==parts[i-1][1] and abs(parts[i][0]-parts[i-1][0])<=10:
        temp.append(parts[i][2])

    else:
        insert_part=''.join(temp)
        r.append(insert_part)
        temp=[parts[i][2]]

        if parts[i][0]<parts[i-1][0]:
            p.append(r)
            r=[]
    '''

f=r'C:/Users/s3309/Desktop/Investment/qrt/202201_2376_AI1_20221016_130720.pdf'
reader = PdfReader(f)
page=start=end=0

#Focus on accounting subjects
while start == 0 and end == 0:
    
    _ = reader.pages[page]
    content=_.extract_text()

    fs=re.findall(r'(?<=重要會計項目之說明).*\d{1,}.*\d{1,}',content)
    if len(fs)!=0:
        fs=fs[0]
        tgt=re.findall(r'\d{1,}',fs)
        start,end=int(tgt[0]),int(tgt[1])
        print(start,end)
    
    page+=1

t=[]
p=[]
temp_p=[]
temp_c=[]
time=[]
time_window=[]
subtitle=[]
category=[]
money=[]
mn=[]
parts=[]
finish=False

cur = rev_found = False

for page in range(start-1,end):

#for page in [32]:
    #if time !=[]:break;
    if finish: break;
    print('page------------------------',page)
    reader.pages[page].extract_text(visitor_text=rev_visitor_body)

#Turn into DataFrame
df_list=[[] for i in range(4)]
for i , n in enumerate(category):
    for j,m in enumerate(category[i]):
        for num in range(len(time_window[i])):
            print(i,j,num)
            df_list[0].append(subtitle[i])
            df_list[1].append(category[i][j])
            df_list[2].append(time_window[i][num])
            df_list[3].append(mn[i][j][num])


data=pd.DataFrame()
for i,col in enumerate(['Subtitle','Category','Time Period','Values']):
    data[col]=df_list[i]


29 49
page------------------------ 28

 

 



~

29

~

 

六、

重要會計

 項

目之說明

  



(

一

)

現金及約當現金

  



 



1

.

本集團往來之金融機構信用品質良好，且本集團與多家金融機構往來以



分散信用風險，預期發生違約之可能性甚低。

  



2.

本集團

將存款期間超過三個月以上及轉供質

 押之

銀行

存款轉列「按攤銷



後成本衡量之金融資產」項下

 ，

請參閱

附註六、

 (

四

)

及

附註八

。

 



(

二

)

透過損益按公允價值衡量之金融資產

 -

流動

 



 



1.

透過損益按公允價值衡量之金融資產

 認列於損益之明細如下：

  



 



2

.

本集團未有

 將透過損

 益按公允價值衡量之金融資產

 提供質押之情

 形

。

 



3

.

相關

透過損益按公允價值衡量之金融資產

 信用風險資訊請詳附註十二、



(

二

)

。

 





111

年

3

月

31

日

110

年

12

月

31

日

110

年

3

月

31

日



庫存現金及週轉金

 4,437

 $      

 

4,666

 $      

 

3,898

 $      

 



支票存款及活期存款

 6,153,416

   

 

10,079,048

  

 

4,612,708

   

 



定期存款

 7,229,429

   

 

8,844,759

   

 

10,353,727

  

 



13,387,282

$ 

 

18,928,473

$ 

 

14,970,333

$ 

 
111年3月31日110年12月31日110年3月31日
庫存現金及週轉金 4,437 $       4,666 $       3,898 $       
支票存款及活期存款 6,153,416    10,079,048   4,612,708    
定期存款 7,229,429    8,844,759    10,3

 impossible to decode XFormObject /Meta149



 

 



~

42

~

 

2.

上述

股份基礎給付

 協議

之

詳細

資訊如下：

  



 



3

.

民國

111

年及

110

年

3

月

31

日

，聲遠精密

 光學股份有限公司

 流通在外



之認股權，履約價格

 均為

10.00

元，加權平均剩餘合約期間

 分別

為

0.78



年

及

1.78

年

。

 



4

.

上開認股選擇權計劃係採

 Black

-

Scholes

 選擇權評價模式估計給與日



認股選擇權之公

 允

價值，

相關資訊如下：

  



 



(

十八

)

股

本

 



1

.

民國

111

年

3

月

31

日

，本公司額定資本額為

 $

9,500,000

 ，分為

950,000



仟股

(

其中

2

50,000

 仟股保留供

 員工認股權憑證

 、

附認股權

 特別股或

 公



司債

行使認股權使用

 )

，實收資本額為

 $6

,3

56

,

88

9

，每股面額

 10

元。本



公司已發行股份之股款均已收訖。

 本公司普通股期初與期末流通在外



股數

於民國

 111

年及

1

10

年

1

月

1

日至

3

月

31

日分別

為

6

34

,

983,886



及

6

3

5,

688,886

 股。

 



2

.

庫藏股

 



(1)

股份收回原因及其數量

 :

 



 



 





加權平均

 加權平均



認股權數量

 履約價格

 認股權數量

 履約價格



員工認股權計畫

 ─

聲遠精密

 (

仟股

)

(

元

)

(

仟股

)

(

元

)



1

月

1

日流通在外認股權

 170

     

 

10.00

$  

 

230

     

 

10.00

$  

 



本期喪失認股權

 -

        

 

-

       

 

-

        

 

-

       

 



3


2022-10-30 20:05:05,410 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to C:\Users\s3309\AppData\Local\Temp\tika-server.jar.md5.
2022-10-30 20:05:06,752 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to C:\Users\s3309\AppData\Local\Temp\tika-server.jar.


: 

: 

In [10]:

for page in reader.pages:
    if "/Annots" in page:
        print(page["/Annots"])
        for annot in page["/Annots"]:
            subtype = annot.get_object()["/Subtype"]
            if subtype == "/Text":
                print(annot.get_object()["/Contents"])

[IndirectObject(1094, 0, 1647117465680), IndirectObject(1095, 0, 1647117465680)]
[IndirectObject(1097, 0, 1647117465680), IndirectObject(1098, 0, 1647117465680)]


In [11]:
%pip install tika


  Created wheel for tika: filename=tika-1.24-py3-none-any.whl size=32893 sha256=25190094eff7227e0809eae2cd4476fc003f4892cb5f338984e4eed1bda14001
  Stored in directory: c:\users\s3309\appdata\local\pip\cache\wheels\ec\76\38\0e4b92d8a3a89cbfff5be03a40c02d15b2072b1b08ebf28d6a
Successfully built tika
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\s3309\AppData\Local\Programs\Python\Python39\dlenv\Scripts\python.exe -m pip install --upgrade pip' command.
